In [1]:
import pandas as pd
import numpy as np
import random
from scipy import stats

# =====================================================
# CONFIGURATION
# =====================================================
DATA_FILE = "Final_Player_Stats.csv"
TEAM_SIZE = {'Batsman': 6, 'Wicketkeeper': 1, 'Bowler': 4}

# --- Longevity Adjustment Constants (Bayesian Shrinkage) ---
# k represents the number of matches where player's stats and league average are equally weighted
# Higher k = more shrinkage towards league average for low-match players
LONGEVITY_CONSTANTS = {
    'batting_k': 20,       # Batsmen need ~20 matches to establish reliability
    'league_batting_avg': 30.0,  # Typical Test cricket batting average
    'bowling_k': 25,       # Bowlers need slightly more matches for consistency
    'league_bowling_avg': 35.0,   # Typical Test cricket bowling average
    'duck_k': 30           # Matches needed to fairly assess duck rate
}

# --- Randomness Control ---
COMPONENT_VARIANCE = 0.05  # 5% variance on component scores
TEAM_VARIANCE = 0.02       # 2% variance on final team score

#Loading Dataset
players_df = pd.read_csv(DATA_FILE)
if players_df.empty:
    print("Player data frame is empty.")

if 'Player_ID' not in players_df.columns:
    players_df['Player_ID'] = players_df.index

players_df['Total_Fielding'] = players_df['Stumpings_Total'] + players_df['Fielding_Catches']

# =====================================================
# HELPER FUNCTIONS
# =====================================================

def percentile_score(value, all_values, min_score=0, max_score=100, lower_is_better=False):
    """
    Convert value to percentile-based score (0-100 range).
    
    Args:
        value: The value to score
        all_values: Reference distribution for percentile calculation
        min_score: Minimum score (default 0)
        max_score: Maximum score (default 100)
        lower_is_better: If True, lower values get higher scores (for bowling avg)
    
    Returns:
        Score between min_score and max_score
    """
    if pd.isna(value) or value < 0:
        return min_score

    valid_values = all_values[all_values.notna() & (all_values >= 0)]
    if len(valid_values) == 0:
        return min_score

    percentile = stats.percentileofscore(valid_values, value, kind='rank')
    
    if lower_is_better:
        percentile = 100 - percentile
        
    score = min_score + (percentile / 100) * (max_score - min_score)
    return np.clip(score, min_score, max_score)


def calculate_consistency_modifier(player_row):
    """
    Calculate batting consistency modifier based on not-outs and ducks.
    
    Longevity-aware duck punishment:
    - More ducks in fewer matches = harsher penalty
    - More ducks in more matches = softer penalty
    
    Returns multiplier between 0.85 and 1.15
    """
    if player_row['Matches_Played'] < 5:
        return 1.0
    
    modifier = 1.0
    
    # Higher not-out ratio = more consistent (positive)
    not_out_ratio = player_row['Not_Outs'] / max(player_row['Matches_Played'], 1)
    modifier += not_out_ratio * 0.08
    
    # LONGEVITY-AWARE DUCK PUNISHMENT
    # Calculate raw duck rate
    raw_duck_rate = player_row['Ducks_Career'] / max(player_row['Matches_Played'], 1)
    
    # Apply Bayesian shrinkage to duck rate
    # Players with fewer matches get their duck rate adjusted towards a typical rate
    k = LONGEVITY_CONSTANTS['duck_k']
    typical_duck_rate = 0.15  # ~15% is typical for Test cricket
    
    longevity_weight = player_row['Matches_Played'] / (player_row['Matches_Played'] + k)
    adjusted_duck_rate = (
        longevity_weight * raw_duck_rate +
        (1 - longevity_weight) * typical_duck_rate
    )
    
    # Penalty based on adjusted duck rate
    # Example: 15 ducks in 50 matches (30% raw) → shrunk to ~22.5% adjusted → larger penalty
    # Example: 15 ducks in 150 matches (10% raw) → stays ~10% adjusted → smaller penalty
    duck_penalty = adjusted_duck_rate * 0.40
    modifier -= duck_penalty
    
    return np.clip(modifier, 0.85, 1.15)


def random_scale(score, max_variance=0.05):
    """
    Apply small random variance to score.
    
    Args:
        score: Base score
        max_variance: Maximum percentage variance (default 5%)
    
    Returns:
        Scaled score
    """
    return score * random.uniform(1 - max_variance, 1 + max_variance)


# =====================================================
# INDIVIDUAL PLAYER SCORE CALCULATION
# =====================================================

def calculate_individual_batting_score(player_row, all_players_df):
    """
    Calculate individual batting score (0-100) with longevity adjustment.
    Applied to ALL players regardless of role.
    """
    # Longevity adjustment (Bayesian shrinkage)
    k = LONGEVITY_CONSTANTS['batting_k']
    league_avg = LONGEVITY_CONSTANTS['league_batting_avg']
    
    longevity_weight = player_row['Matches_Played'] / (player_row['Matches_Played'] + k)
    adjusted_batting_avg = (
        longevity_weight * player_row['Batting_Average'] +
        (1 - longevity_weight) * league_avg
    )
    
    # Component scores (0-100 range)
    avg_score = percentile_score(adjusted_batting_avg, all_players_df['Batting_Average'], 0, 100)
    runs_score = percentile_score(player_row['Total_Runs'], all_players_df['Total_Runs'], 0, 100)
    centuries_score = percentile_score(player_row['Centuries'], all_players_df['Centuries'], 0, 100)
    fifties_score = percentile_score(player_row['Fifties'], all_players_df['Fifties'], 0, 100)
    
    # Consistency modifier (includes longevity-aware duck punishment)
    consistency = calculate_consistency_modifier(player_row)
    
    # Weighted combination
    batting_score = (
        avg_score * 0.35 +
        runs_score * 0.25 +
        centuries_score * 0.20 +
        fifties_score * 0.20
    ) * consistency
    
    return np.clip(batting_score, 0, 100)


def calculate_individual_bowling_score(player_row, all_players_df):
    """
    Calculate individual bowling score (0-100) with longevity adjustment.
    Applied to ALL players regardless of role.
    
    Better bowling average (lower) over more matches = better player
    """
    # Longevity adjustment (Bayesian shrinkage)
    k = LONGEVITY_CONSTANTS['bowling_k']
    league_avg = LONGEVITY_CONSTANTS['league_bowling_avg']
    
    longevity_weight = player_row['Matches_Played'] / (player_row['Matches_Played'] + k)
    adjusted_bowling_avg = (
        longevity_weight * player_row['Bowling_Average'] +
        (1 - longevity_weight) * league_avg
    )
    
    # Component scores (0-100 range, lower bowling average is better)
    avg_score = percentile_score(adjusted_bowling_avg, all_players_df['Bowling_Average'], 
                                  0, 100, lower_is_better=True)
    wickets_score = percentile_score(player_row['Wickets_Total'], all_players_df['Wickets_Total'], 0, 100)
    five_wickets_score = percentile_score(player_row['Five_Wickets'], all_players_df['Five_Wickets'], 0, 100)
    matches_score = percentile_score(player_row['Matches_Played'], all_players_df['Matches_Played'], 0, 100)
    
    # Weighted combination
    bowling_score = (
        avg_score * 0.35 +
        wickets_score * 0.30 +
        five_wickets_score * 0.20 +
        matches_score * 0.15
    )
    
    return np.clip(bowling_score, 0, 100)


def calculate_individual_fielding_score(player_row, all_players_df):
    """
    Calculate individual fielding score (0-100).
    Applied to ALL players.
    """
    catches_score = percentile_score(player_row['Fielding_Catches'], 
                                     all_players_df['Fielding_Catches'], 0, 100)
    stumpings_score = percentile_score(player_row['Stumpings_Total'], 
                                       all_players_df['Stumpings_Total'], 0, 100)
    
    # Wicketkeepers get bonus for stumpings
    if player_row['Player_Category'] == 'Wicketkeeper':
        stumpings_score = np.clip(stumpings_score * 1.2, 0, 100)
    
    # Weighted combination
    fielding_score = (
        catches_score * 0.65 +
        stumpings_score * 0.35
    )
    
    return np.clip(fielding_score, 0, 100)


def calculate_individual_experience_score(player_row, all_players_df):
    """
    Calculate individual experience score (0-100).
    Applied to ALL players.
    
    FIXED: Now only uses Matches_Played to avoid double-counting
    (Runs already in Batting, Wickets already in Bowling)
    """
    # Pure career longevity metric - no overlap with other scores
    matches_score = percentile_score(player_row['Matches_Played'], 
                                     all_players_df['Matches_Played'], 0, 100)
    
    experience_score = matches_score  # 100% based on matches played
    
    return np.clip(experience_score, 0, 100)


# =====================================================
# TEAM SELECTION - PURELY RANDOM 
# =====================================================

def select_players_random(pool, count):
    """
    Select players purely randomly from the pool.
    No strategy bias, no history tracking.
    
    Args:
        pool: DataFrame of available players for this role
        count: Number of players to select
    
    Returns:
        DataFrame of selected players
    """
    if len(pool) < count:
        print(f"⚠️ Warning: Not enough players in pool. Need {count}, have {len(pool)}")
        return pool.sample(len(pool), random_state=random.randint(1, 99999))
    
    selected_players = pool.sample(count, random_state=random.randint(1, 99999))
    return selected_players.reset_index(drop=True)


# =====================================================
# SELECT TEAM - PURELY RANDOM (No Strategy, No History)
# =====================================================
print("\n🎯 TEAM SELECTION")

batsmen_pool = players_df[players_df['Player_Category'] == 'Batsman']
keepers_pool = players_df[players_df['Player_Category'] == 'Wicketkeeper']
bowlers_pool = players_df[players_df['Player_Category'] == 'Bowler']

team_batsmen = select_players_random(batsmen_pool, TEAM_SIZE['Batsman'])
team_keeper = select_players_random(keepers_pool, TEAM_SIZE['Wicketkeeper'])
team_bowlers = select_players_random(bowlers_pool, TEAM_SIZE['Bowler'])

team_df = pd.concat([team_batsmen, team_keeper, team_bowlers]).reset_index(drop=True)

# Verify no duplicate players in same team
if len(team_df['Player_ID'].unique()) != 11:
    print("⚠️ Warning: Duplicate players detected in same team. This should not happen!")
    team_df = team_df.drop_duplicates(subset=['Player_ID'], keep='first')

if len(team_df) != 11:
    print(f"❌ Error: Could only select {len(team_df)} unique players.")

# =====================================================
# CALCULATE INDIVIDUAL SCORES FOR ALL 11 PLAYERS
# =====================================================
print("📊 Calculating individual player scores...")

team_df['Individual_Batting_Score'] = team_df.apply(
    lambda row: calculate_individual_batting_score(row, players_df), axis=1
)
team_df['Individual_Bowling_Score'] = team_df.apply(
    lambda row: calculate_individual_bowling_score(row, players_df), axis=1
)
team_df['Individual_Fielding_Score'] = team_df.apply(
    lambda row: calculate_individual_fielding_score(row, players_df), axis=1
)
team_df['Individual_Experience_Score'] = team_df.apply(
    lambda row: calculate_individual_experience_score(row, players_df), axis=1
)

# Calculate total individual score with REALISTIC WEIGHTS
# Batting & Bowling are most important (35% each)
# Fielding & Experience are valuable but secondary (15% each)
team_df['Individual_Total_Score'] = (
    team_df['Individual_Batting_Score'] * 0.35 +
    team_df['Individual_Bowling_Score'] * 0.35 +
    team_df['Individual_Fielding_Score'] * 0.15 +
    team_df['Individual_Experience_Score'] * 0.15
)

# =====================================================
# AGGREGATE TO TEAM COMPONENT SCORES (0-100 each)
# =====================================================

# --- BATTING SCORE ---
# Use top 6-7 batting performers (batsmen + keeper)
batting_contributors = team_df[team_df['Player_Category'].isin(['Batsman', 'Wicketkeeper'])].copy()
batting_contributors = batting_contributors.nlargest(7, 'Individual_Batting_Score')

# Weight top batters more heavily
weights_batting = [1.25, 1.20, 1.15, 1.10, 1.05, 1.0, 1.0][:len(batting_contributors)]
team_batting_score = np.average(batting_contributors['Individual_Batting_Score'].values, 
                                 weights=weights_batting)
team_batting_score = random_scale(team_batting_score, COMPONENT_VARIANCE)
team_batting_score = np.clip(team_batting_score, 0, 100)

# --- BOWLING SCORE ---
# Use top 5 bowling performers (accounts for 5th bowler usage)
bowling_contributors = team_df.nlargest(5, 'Individual_Bowling_Score')

# Weight top bowlers more heavily
weights_bowling = [1.30, 1.20, 1.10, 1.0, 1.0][:len(bowling_contributors)]
team_bowling_score = np.average(bowling_contributors['Individual_Bowling_Score'].values, 
                                 weights=weights_bowling)
team_bowling_score = random_scale(team_bowling_score, COMPONENT_VARIANCE)
team_bowling_score = np.clip(team_bowling_score, 0, 100)

# --- FIELDING SCORE ---
# All 11 players contribute equally
team_fielding_score = team_df['Individual_Fielding_Score'].mean()
team_fielding_score = random_scale(team_fielding_score, COMPONENT_VARIANCE)
team_fielding_score = np.clip(team_fielding_score, 0, 100)

# --- EXPERIENCE SCORE ---
# All 11 players, but weight experienced players more
exp_sorted = team_df.sort_values('Individual_Experience_Score', ascending=False)
weights_exp = [1.20, 1.15, 1.10, 1.05, 1.0] + [0.95] * 6
weights_exp = weights_exp[:len(exp_sorted)]

team_experience_score = np.average(exp_sorted['Individual_Experience_Score'].values, 
                                    weights=weights_exp)
team_experience_score = random_scale(team_experience_score, COMPONENT_VARIANCE)
team_experience_score = np.clip(team_experience_score, 0, 100)

# =====================================================
# TEAM TOTAL SCORE (0-1000)
# Sum of all 11 players' individual totals, scaled
# =====================================================

# Raw team total = sum of all individual player totals
raw_team_total = team_df['Individual_Total_Score'].sum()

# Scale from raw sum to 0-1000
# New range: 11 players × (0-100 weighted) = 0-100 per player
# With weights: 0.35 + 0.35 + 0.15 + 0.15 = 1.0, so max = 100 per player
min_possible = 0
max_possible = 11 * 100  # 1100 (max total for all 11 players)

team_total_score = (raw_team_total / max_possible) * 1000
team_total_score = np.clip(team_total_score, 0, 1000)

# Apply small random variance to final score
team_total_score = random_scale(team_total_score, TEAM_VARIANCE)

# =====================================================
# GAME VARIABLES — HOME/AWAY + LUCK
# =====================================================
home_or_away = random.choice(["Home", "Away"])
luck_factor = random.randint(1, 6)

# Luck effect mapping (1=very unlucky, 6=very lucky)
luck_effect_map = {1: -0.05, 2: -0.02, 3: 0, 4: 0.02, 5: 0.05, 6: 0.10}

# Apply home/away advantage
if home_or_away == "Home":
    team_total_score *= 1.05  # 5% boost for home
else:
    team_total_score *= 0.97  # 3% reduction for away

# Apply luck factor
team_total_score *= (1 + luck_effect_map[luck_factor])

# Final clipping
team_total_score = np.clip(team_total_score, 0, 1000)

# =====================================================
# OUTPUT
# =====================================================
team_summary = {
    "Team_Selection": "PURELY RANDOM ",
    "Team_Batting_Score": round(team_batting_score, 2),
    "Team_Bowling_Score": round(team_bowling_score, 2),
    "Team_Fielding_Score": round(team_fielding_score, 2),
    "Team_Experience_Score": round(team_experience_score, 2),
    "Home_Away": home_or_away,
    "Luck_Factor": luck_factor,
    "Team_Total_Score": round(team_total_score, 2)
}

print("\n" + "="*60)
print("  CRICKET TEAM GENERATION  ")
print("="*60)
for k, v in team_summary.items():
    print(f"{k.replace('_', ' '):<30}: {v}")

print("\n" + "-"*60)
print("SELECTED PLAYERS WITH INDIVIDUAL SCORES (0-100 scale)")
print("-"*60)

output_df = team_df[[
    'Player_Name', 'Player_Category', 'Matches_Played',
    'Individual_Batting_Score', 'Individual_Bowling_Score',
    'Individual_Fielding_Score', 'Individual_Experience_Score',
    'Individual_Total_Score'
]].round(2)

print(output_df.to_string(index=False))

print("\n" + "-"*60)
print("PLAYER STATISTICS (Raw Data)")
print("-"*60)
stats_df = team_df[[
    'Player_Name', 'Batting_Average', 'Total_Runs', 'Ducks_Career',
    'Wickets_Total', 'Bowling_Average', 'Fielding_Catches', 'Stumpings_Total'
]].round(2)
print(stats_df.to_string(index=False))

print("\n" + "="*60)
print(f"✅ Total individual scores sum: {team_df['Individual_Total_Score'].sum():.2f}")
print(f"✅ Scaled to team total (0-1000): {team_total_score:.2f}")
print(f"✅ Average player score: {team_df['Individual_Total_Score'].mean():.2f}")
print("="*60)


🎯 TEAM SELECTION
📊 Calculating individual player scores...

  CRICKET TEAM GENERATION  
Team Selection                : PURELY RANDOM 
Team Batting Score            : 65.99
Team Bowling Score            : 50.13
Team Fielding Score           : 46.58
Team Experience Score         : 47.82
Home Away                     : Away
Luck Factor                   : 4
Team Total Score              : 486.99

------------------------------------------------------------
SELECTED PLAYERS WITH INDIVIDUAL SCORES (0-100 scale)
------------------------------------------------------------
 Player_Name Player_Category  Matches_Played  Individual_Batting_Score  Individual_Bowling_Score  Individual_Fielding_Score  Individual_Experience_Score  Individual_Total_Score
    GR Stead         Batsman               5                     60.92                     34.87                      40.66                        41.76                   45.89
  LS Birkett         Batsman               4                     57.18 